In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# KNN

import csv
import random
import math
import operator
from sklearn.preprocessing import MinMaxScaler

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    
    with open(filename, 'rt') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        dataset2 = dataset
        scaler = MinMaxScaler()
        scaler.fit(dataset)
        MinMaxScaler(copy=True, feature_range=(0, 1))
        dataset = scaler.transform(dataset)
        for a in range(len(dataset)):
            dataset[a][10] = dataset2[a][10]
        for x in range(0,len(dataset)):
            for y in range(9):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

# length itu jumlah attribut atau jumlah grafik
def euclideanDistance(instance1, instance2, length, chrom2):
    distance = 0
    for x in range(1,length): # cari jarak dari titik data test ke data training tiap grafik
        if chrom2[x]==1: # kalau flag kromosom 1 alias dipakek, maka jarak dicari
            distance += pow((float(instance1[x])-float(instance2[x])), 2)
    return math.sqrt(distance)

# iter getNeigh data test ke - x
#                trainingSet, testSet[x], k, chrom
def getNeighbors(trainingSet, testInstance, k, chrom2): #testinstance itu attribut yang pengen ditest. anggepannya grafik
    distances = []
    length = len(testInstance)-1 # jumlah attribut
    for x in range(len(trainingSet)): # mulai memetakan jarak dari titik data test ke titik data training
        dist = euclideanDistance(testInstance, trainingSet[x], length, chrom2)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
#     print("banyak tetangga: ")
#     print(len(neighbors))
    return neighbors

# neigh = array grafik jarak attribut
def getResponse(neighbors):
	classVotes = {}
# 	print("Panjang neigh=" + str(len(neighbors)))
	for x in range(len(neighbors)): # jumlah array/attribut yang dipakai/kromosom
		response = neighbors[x][-1] # -1 soalnya id itu grafik -1
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
# 	print(sortedVotes[0][0])
	return sortedVotes[0][0]

# lebih sedikit memprediksikan attribut/fitur/kromosom
# maka akurasi makin tinggi
# pred 10
# pred2 5
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
# 		print("testset: " + str(testSet[x][-1]))
# 		print("prediksi: " + str(predictions[x]))
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0
	

# kromosom nya dilempar ke fitness value
# knn_chrom=[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
def fitnessValue(chrom):
    random.seed(2)
    trainingSet=[]
    testSet=[]
    split = 0.8
    loadDataset('../input/dataset/glass.data', split, trainingSet, testSet)
    predictions=[]
    k = 3
#     print("banyak testset: ")
#     print(len(testSet))
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k, chrom) # kalau kromosomnya 1 alias atribut itu dipakai. neigh bentuknya array
        result = getResponse(neighbors)
        predictions.append(result)
#         print("ITERASI testSet ke: " + str(x))
#         print("banyak neigh:" + str(len(neighbors)))
#         print("banyak prediksi: " + str(len(predictions)))
#     print("banyak prediksi: ")
#     print(len(predictions))
    accuracy = getAccuracy(testSet, predictions)
    return repr(accuracy)

In [ ]:
# GA
import numpy

# buat array populasi yang terdiri dari N individu
def generatePop(n):
	chromAsli=[]
	for x in range(n):
		temp=[] # 1 baris kekanan / kromosom
		temp2=[] # jumlah baris
		for x in range(10):
			temp.append(numpy.random.randint(1))
		temp2.append(temp) # [kromosom]
		fitness=fitnessValue(temp) # cari akurasi dari satu baris kromosom pakek KNN
		temp2.append(fitness) # akurasinya dimasukin setelah kromosomnya [kromosom, akurasi]
		chromAsli.append(temp2) # kumpulan array [kromosom, akurasi] dalam satu populasi
	return chromAsli

# n = genNum / banyaknya individu
def eliteChild(chrom2,n):
	a=sorted(chrom2,key=lambda l:l[1], reverse=True) # sort desc
	bestVal=a[0][1]
	bestFt2=a[0][0]
	next2=[]
	chrom3=[]
	for x in range(2): # 2 elite child
		next2.append(a[x][0])
	for x in range(2,n): # sisa individu
		chrom3.append(a[x])
	return chrom3,next2,bestVal,bestFt2
#             chrom2,nextGen,bestFitness,bestFt2

# buat milih kromosom yang paling bagus
def tournament(chrom2):
    best=[]
    for x in range(2):
        a=numpy.random.randint(0,7)
        best.append(chrom2[a])
    bestOne=best[0]
    if(best[0][1]<best[1][1]):
        bestOne=best[1]
    return bestOne[0]

# mutasi kromosom dari elite child
#               chrom2,0.3,nextGen,genNum
def getMutation(chrom2,pm,next2,n): # next2 itu 2 elite child
    rng=n-len(next2) # range = jumlah individu - 2 elite child
    for i in range(rng): # pasti 8
        a=tournament(chrom2) # hasil kromosom paling bagus akurasinya ditandai sama 1
        rd=[] # rd itu XOR
        for x in range(10):
            ru=numpy.random.uniform(0.0, 1.0) # kenapa tadi tiba" akurasi dan kromosomnya berubah, karena ada fungsi random
            if ru<=pm:
                rd.append(1) # ini flag buat tandain XOR nya. 1 True
            else:
                rd.append(0) # 0 False
        if a==rd:
            a=tournament(chrom2)
        result=toXor(a,rd)
        next2.append(result)
    return next2

def toXor(x1,x2):
	xorRes=[]
	for x in range(len(x1)):
		if x1[x]==x2[x]:
			xorRes.append(0)
		else:
			xorRes.append(1)
	return xorRes

# pr = split
# rng = range
def getCrossOver(chrom2,pr,next2):
	rng=int(pr*len(chrom2))
	for x in range(rng):
		a=tournament(chrom2)
		b=tournament(chrom2)
		if a==b:
			result=a
		else:
			result=toXor(a,b)
		next2.append(result)
	return next2

# Langkah:
# 1. Cari 2 Elite Child
# 2. Elite Child nya di Crossover
# 3. Mutation

# perubahan kromosom ada di Mutasi
def getGeneration(chrom2,idx):
	chrom2,nextGen,bestFitness,bestFt2=eliteChild(chrom2,genNum) # Cari 2 individu dengan Fitness Terbaik
	nextGen=getCrossOver(chrom2,0.8,nextGen) # 0,8 itu split
	nextGen=getMutation(chrom2,0.3,nextGen,genNum) # 0,3 Pm
	print('Generation {} {:.3f}'.format(idx+1,float(bestFitness)),end='%\n') # generasi ke - x
	return nextGen,bestFitness,bestFt2



In [ ]:
# Satu generasi, isinya ada 15 individu (variabel genNum)
# Batas generasi ada 120 (variabel Limit)

genNum=15 # Banyaknya individu
knn_chrom=[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Ambil fitnessValue/Akurasi dari KNN 
knn_acc=fitnessValue(knn_chrom)
# print("Akurasi KNN= " + str(knn_acc))

limit=5 # Jumlah generasi
stall=3 # Batas jumlah generasi ketika akurasinya sama

# perubahan akurasi terakhir ada di generasi ke X
# stall akan membatasi reproduksi generasi hingga X + STALL

# deklarasi kromosom awal dari 15 individu
chrom=generatePop(genNum)

# Deklarasi Variabel GA
counter=0
newFit=0.0 # Nilai akurasi setelahnya -> paling bagus
oldFit=0.0 # Nilai akurasi sebelumnya

# APA HUBUNGANNYA KNN DAN GA?
# Pertama kali menggunakan KNN dengan nilai K tertentu dan Split tertentu, lalu dicari akurasinya
# Setelah ketemu nilai akurasi dengan KNN, dibuatlah sebuah Populasi yang terdiri dari 15 individu dengan 10 attribut (kromosom) random dipakai tidaknya
# dengan jumlah Generasi (limit) dan batas (stall) tertentu
# Kemudian, akan dilakukan KNN (nilai K dan Split sama dengan awal pertama kali KNN) pada tiap individu dengan kromosom yang random dipakai atau tidaknya
# Pengaruh kromosom dipakai atau tidaknya pada KNN adalah jarak Euclidian yang akan tidak ada atau ada.
# Jika jarak euclidian tidak ada, fitur dan parameter akan lebih sedikit, sehingga akurasi lebih tinggi
# Pilihan  fitur  digunakan  untuk  menemukan fitur yang paling mewakili kelas pada dataset tersebut

In [ ]:
for x in range(limit): # iterasi setiap generasi
    oldFit=newFit
    
    # AMBIL newChrom, newFit, sama bestFit untuk generasi berikutnya (new)
    newChrom,newFit,bestFt = getGeneration(chrom,x) #ambil generasi yang kromosomnya baru dan terbaik (SETELAH GA)
    newFit=float(newFit)
    temp2=[]
#     print("ITER X ke " + str(x))
    for y in range(genNum): # iterasi sampek 15 individu
        temp=[]
        temp.append(newChrom[y]) # buat array isinya kromosom baru untuk tiap individu dari Generasi Baru
#         print("iter y ke " + str(y))
#         print("TEMP APPEND NEW CHROM")
#         print(temp, sep = "\n")
#         print("\n")
#         print("============ GA ============")
        f=fitnessValue(newChrom[y]) # cari fitnessvalue dari tiap kromosom baru
        temp.append(f)
        temp2.append(temp)
#         print("TEMP APPEND FITNESS VALUE")
#         print(temp, sep = "\n")
#         print("\n")
#         print("TEMP2 APPEND TEMP")
#         print(temp2, sep = "\n")
#         print("\n")
    chrom=temp2 # kromosom diubah ke array kromosom dan fitness baru (temp2)
    diff=newFit - oldFit
#     print("difference: " + str(diff))
    if diff<=0.0000001: # jika akurasinya sama, counternya nambah
        counter+=1
#         print("TAMBAH counter=" + str(counter))
    else:
    	counter=0
#     print("counter=" + str(counter))
    if counter==stall: # GA nya diiterasi sampek stall
    	break
    
print("Accuracy using K-NN without GA: ",end='')
print('{:.3f}'.format(float(knn_acc)),end='')
print("%\nAccuracy using K-NN with GA:",end='')
print('{:.3f}'.format(float(newFit)),end='%')
print("\nUsed features:", bestFt)